# Sentiment classification

In [1]:
!pip install perceiver-io[text]==0.3.0

     |████████████████████████████████| 409 kB 13.8 MB/s 
     |████████████████████████████████| 584 kB 39.1 MB/s 
     |████████████████████████████████| 135 kB 45.5 MB/s 
     |████████████████████████████████| 248 kB 48.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 65.7 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 136 kB 59.1 MB/s 
     |████████████████████████████████| 1.1 MB 57.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 46.7 MB/s 
     |████████████████████████████████| 144 kB 54.3 MB/s 
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307252 sha256=dd82119f17a3f4bdc89a98e8454598078162cc0d68f83d8d9bc633262496d1be
  Stored in directory: /root/.cache/pi

In [2]:
# Download trained tokenizer
!wget https://raw.githubusercontent.com/krasserm/perceiver-io/main/.cache/imdb-tokenizer-10003.json

--2022-05-12 04:28:13--  https://raw.githubusercontent.com/krasserm/perceiver-io/main/.cache/imdb-tokenizer-10003.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136806 (134K) [text/plain]
Saving to: ‘imdb-tokenizer-10003.json’

imdb-tokenizer-1000 100%[===================>] 133.60K  --.-KB/s    in 0.01s   

2022-05-12 04:28:13 (9.40 MB/s) - ‘imdb-tokenizer-10003.json’ saved [136806/136806]



In [3]:
# Download checkpoints
!wget -nc -O logs.zip https://martin-krasser.com/perceiver/logs-update-4.zip
!unzip -qo logs.zip

connected.
HTTP request sent, awaiting response... 200 OK
Length: 44278987 (42M) [application/zip]
Saving to: ‘logs.zip’

logs.zip            100%[===================>]  42.23M  13.6MB/s    in 3.1s    

2022-05-12 04:28:30 (13.6 MB/s) - ‘logs.zip’ saved [44278987/44278987]



In [4]:
import torch

from perceiver.data.text import TextPreprocessor
from perceiver.model.text.classifier import LitTextClassifier

In [5]:
imdb_preproc = TextPreprocessor(tokenizer_path='imdb-tokenizer-10003.json')

In [6]:
ckpt_path = 'logs/seq_clf/version_1/checkpoints/epoch=017-val_loss=0.254.ckpt'

model = LitTextClassifier.load_from_checkpoint(ckpt_path, clf_ckpt=None).model
model.eval();

In [7]:
text_batch = [
    "I've seen this movie yesterday and it was really boring",
    "I can recommend this movie to all fantasy movie lovers"
]

In [8]:
with torch.no_grad():
    logits = model(*imdb_preproc.preprocess_batch(text_batch))
    preds = logits.argmax(dim=1)

In [9]:
for text, pred in zip(text_batch, preds):
    print(f'{text} (positive = {pred == 1})')

I've seen this movie yesterday and it was really boring (positive = False)
I can recommend this movie to all fantasy movie lovers (positive = True)
